# Workflow to calculate the quantum efficiency of a polycrystalline material

In [28]:
import subprocess
import nglview as nv

from ase.calculators.castep import Castep
from ase.atoms import Atoms
from ase.io import read, write
from ase.visualize import view
from ase.build import surface
import ase.calculators.castep
import ase.io.castep

from pymatgen.ext.matproj import MPRester
from pymatgen.core import Lattice, Structure, Molecule
from pymatgen.core.surface import SlabGenerator, generate_all_slabs
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.analysis.wulff import WulffShape
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.electronic_structure.bandstructure import BandStructure
from pymatgen.symmetry.kpath import KPathLatimerMunro
from pymatgen.vis.structure_chemview import quick_view

from wulffpack import SingleCrystal

from functions import *

I recommend using pymatgen to generate the bulk/slab and then convert it to an ASE atom object. The advantages of this approach are:
- pymatgen has a better bulk/slab generation algorithm 
- castep can link a calculator to the Atom object


#### Download and save .cif file for necessary materials

In [ ]:
import json
api_key = ''
with MPRester(api_key) as m:
    results = m.get_dos_by_material_id("mp-30")
#print(type(results))
#res_json = results.as_dict()
with open('./structures/jsons/DOS/Cu_bulk_dos_ref.json', 'w') as f:
    json.dump(results.as_dict(), f)

#### Generate the bulk structure and bulk input

Full Formula (Cu1)
Reduced Formula: Cu
abc   :   2.560619   2.560619   2.560619
angles:  60.000000  60.000000  60.000000
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cu      0    0    0
11.871889685682168
2.217561 0.000000 1.280309
0.739187 2.090737 1.280309
0.000000 0.000000 2.560619
[Element Cu]
[PeriodicSite: Cu (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]]
Atoms(symbols='Cu', pbc=True, cell=[[2.217561034131073, 0.0, 1.2803094600000002], [0.7391870113770244, 2.090736593238846, 1.2803094600000002], [0.0, 0.0, 2.56061892]])
Full Formula (Cu1)
Reduced Formula: Cu
abc   :   2.560619   2.560619   2.560619
angles:  60.000000  60.000000  60.000000
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cu      0    0    0


NGLWidget()

In [15]:
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
#view(AseAtomsAdaptor().get_atoms(bulk))
bulk_seed = 'Cu_bulk_vic_thesis'
bulk_castep_options_template = {
    'directory': f"./structures/{bulk_seed}",
    'label': bulk_seed,
    # Param File Instructions
    'task': 'Singlepoint', #Choose: SinglePoint, BandStructure, Spectral
    'spectral_task' : 'DOS', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All 
    'xc_functional': 'PBE',
    'energy_cutoff': 600,
    'opt_strategy': 'Speed',
    'fix_occup' : False,
    'spin_polarized': False,
    'max_scf_cycles': '100',
    'write_potential': False,
    'write_density': False,
    'extra_bands': True,
    'number_extra_bands': 20,
    #Cell File Instructions
    'kpoints': (16,16,16),
    'snap_to_symmetry': False,
    'fix_all_cell': False,
    'continuation': False,
    'bandstruct_path': 'GXWKGLUWLK',
    'bandstruct_kpt_dist': 0.0184,
    'spectral_kpt_grid': (10,10,10),
    'calculate_pdos': True 
}
bulk_PBS_options = {
    'seed_name': bulk_seed,
    'tasks_seeds': [['singlepoint', bulk_seed], ['Optados', bulk_seed]], #Choose one or multiple (carefully!): SinglePoint, BandStructure, Spectral, OptaDOS
    'queue': 'debug',
    'castep_version': 'castep_19' #choose from castep_19, castep_18, castep_18_mod
}
OptaDOS_options = {
    'seed_name': bulk_seed,
    'optados_task': 'pdos', # Choose: dos(default), compare_dos, compare_jdos, jdos, pdos, optics, core, all
    'broadening': 'adaptive', #Choose: adaptive(default), fixed, linear
    'iprint': '1', #Choose: 1 - bare minimum, 2 - with progress reports, 3 - fulld debug output
    'efermi': 'optados', #Choose: optados - recalculate, file - read from CASTEP file, insulator - count filled bands, float - supplied by user
    'dos_spacing': '0.001', #DOS spacing in unit (default: eV): default - 0.1
    'pdos': 'angular' #Choose: angular, species_ang, species, sites or more detailed descriptions such as: 
    #PDOS : sum:Si1-2(s) - sum of s-chnnls on 2 Si atms (1 proj), 
    #PDOS : Si1;Si2(s) - DOS on Si atom 1 and DOS on s-channel of Si atom 2 (2 proj) 
}
generate_castep_input(calc_struct=AseAtomsAdaptor().get_atoms(bulk), **bulk_castep_options)
generate_qsub_file(**bulk_PBS_options)
generate_optados_input(OptaDOS_options)

NameError: name 'bulk_castep_options' is not defined

#### Generate the surface structure

In [3]:
def get_adjusted_kpts(original, new, kpt = [1,1,1]):
    orig_cell  = original.cell.cellpar()
    
    new_cell  = new.cell.cellpar()
    if not all(abs(new_cell[index] - elem) > 0.001 for index,elem in enumerate(orig_cell[3:])):
        warnings.warn(f'!WARNING! The two cells axes are not in good agreemen, recheck the output!')
    new = [int(orig_cell[0]//new_cell[0]*kpt[0]), int(orig_cell[1]//new_cell[1]*kpt[1]),int(orig_cell[2]//new_cell[2]*kpt[2])]
    for index, item in enumerate(new):
        if item == 0:
            new[index] = 1
    return new;

In [27]:
bulk = Structure.from_file(filename="Cu_metal_fcc.cif")
bulk_ase = AseAtomsAdaptor().get_atoms(bulk)
print(bulk_ase)
print(bulk)
surface = ase.build.surface(lattice = bulk_ase, indices = (1,1,1), layers = 5, vacuum=15, tol=1e-10, periodic=True)
quick_view(bulk)

Atoms(symbols='Cu', pbc=True, cell=[[2.217561034131073, 0.0, 1.2803094600000002], [0.7391870113770244, 2.090736593238846, 1.2803094600000002], [0.0, 0.0, 2.56061892]])
Full Formula (Cu1)
Reduced Formula: Cu
abc   :   2.560619   2.560619   2.560619
angles:  60.000000  60.000000  60.000000
Sites (1)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Cu      0    0    0


RuntimeError: To use quick_view, you need to have chemview installed.

In [9]:

def create_slab_layer_convergence(structure, indices, min, max, seed, *cutoff):
    layers = [i for i in range(min, max+1)]
    castep_opt_template = {
        'directory': f"./structures/{seed}/",
        'label': seed,
        # Param File Instructions
        'task': 'Singlepoint', #Choose: SinglePoint, BandStructure, Spectral
        'spectral_task' : 'DOS', #Choose if task is Spectral: DOS, BandStructure, Optics, CoreLoss, All 
        'xc_functional': 'PBE',
        'energy_cutoff': 566,
        'opt_strategy': 'Speed',
        'fix_occup' : False,
        'spin_polarized': False,
        'max_scf_cycles': '100',
        'write_potential': False,
        'write_density': False,
        'extra_bands': True,
        'number_extra_bands': 20,
        #Cell File Instructions
        'kpoints': [9,9,9],
        'snap_to_symmetry': False,
        'fix_all_cell': False,
        'continuation': False,
        'bandstruct_path': 'GXWKGLUWLK',
        'bandstruct_kpt_dist': 0.0184,
        'spectral_kpt_grid': (10,10,10),
        'calculate_pdos': True 
    }
    PBS_options = {
        'seed_name': seed,
        'tasks_seeds': [], #Choose one or multiple (carefully!): SinglePoint, BandStructure, Spectral, OptaDOS
        'queue': 'debug',
        'castep_version': 'castep_19' #choose from castep_19, castep_18, castep_18_mod
    }
    for i in layers:
        surface = ase.build.surface(lattice = structure, indices =indices, layers = i, vacuum=15, tol=1e-10, periodic=True)
        temp_opt = castep_opt_template
        temp_opt['directory'] = f"./structures/{seed}"
        temp_opt['label'] = f"{seed}_{i}L"
        temp_opt['kpoints'] = get_adjusted_kpts(structure, surface, [9,9,9])
        PBS_options['tasks_seeds'].append(['singlepoint',temp_opt['label']])
        if cutoff != None:
            temp_opt['cutoff'] = cutoff
        generate_castep_input(calc_struct=surface, **temp_opt)
    generate_qsub_file(**PBS_options)
    return;

In [6]:
surface_111 = ase.build.surface(lattice = AseAtomsAdaptor().get_atoms(bulk), indices = (1,1,1), layers = 5, vacuum=10, tol=1e-10, periodic=True)#
view(surface_111)#, repeat=(5,5,1))
super_surf_111 = surface_111.repeat((3,3,1))
#view(super_surf_111)
#surface_100 = ase.build.surface(lattice = bulk, indices = (1,0,0), layers = 5, vacuum=10, tol=1e-10, periodic=True)#
#view(surface, repeat=(5,5,1))
#super_surf_100 = surface_100.repeat((3,3,1))
#view(super_surf_100)
#surface_110 = ase.build.surface(lattice = bulk, indices = (1,1,0), layers = 5, vacuum=10, tol=1e-10, periodic=True)#
#view(surface, repeat=(5,5,1))
#super_surf_110 = surface_110.repeat((3,3,1))
#view(super_surf_110)


#ASE Atoms object for calculation
surf_struct = surface_111
surf_seed = 'Cu_111' # optional

## Options for CASTEP files
surf_castep_options = {
    'directory': './structures/{}'.format(surf_seed),
    'label': surf_seed,
    # Param File Instructions
    'task': 'SinglePoint',
    'xc_functional': 'PBE',
    'energy_cutoff': 600,
    'opt_strategy': 'Speed',
    'fix_occup' : 'FALSE',
    'spin_polarized': 'FALSE',
    'max_scf_cycles': '100',
    'write_potential': False,
    'write_density': False,
    'extra_bands': True,
    'number_extra_bands': 12,
    #Cell File Instructions
    'kpoints': (36,36,1),
    'snap_to_symmetry': False,
    'fix_all_cell': 'FALSE',
    'continuation': False
}

surf_PBS_options = {
    'seed_name': surf_seed,
    'task' : 'SinglePoint',
    'queue': 'short',
    'program': 'castep_19',
    'bandstructure': False
}

#generate_castep_input(calc_struct=surf_struct, **surf_castep_options)
#generate_qsub_file(**surf_PBS_options)


#### Run the calculations

In [ ]:
#folder_change_command = 'cd {}'.format(seed)
#submit_command = 'qsub {}.qsub'.format(seed)

#subprocess.call(folder_change_command)
#subprocess.call(submit_command)
#subprocess.call('cd ..')

#### Extract the final energies

In [ ]:
bulk_tot_energy, bulk_fermi = get_energies(seed=bulk_seed)
surf_111_tot_energy, surf_111_fermi = get_energies(seed=surf_seed)
#print(bulk_tot_energy, surf_111_tot_energy)

#### Calculate the surface energy

In [ ]:
surf_energy_111 = calc_surface_energy(bulk_tot_energy, surf_111_tot_energy,len(surface_111.numbers),surface_111.cell.volume / surface_111.cell[2][2])
factor = 16.02176565
print('The surface energy of the {} surface is: '.format(surf_seed) + str(surf_energy_111) + ' eV/Angstrom^2.\n' + 'This is equal to: ' + str(surf_energy_111*factor) + ' J/m^2')


#### Get the % of each surface from a  Wulff construction

In [ ]:
mat_structure = bulk
facets_energies = {
    (1,0,0) : 0.1161,
    (1,1,0) : 0.1211,
    (1,1,1) : 0.1055
}
facets_fractions = get_wulff_fractions(AseAtomsAdaptor().get_atoms(bulk), facets_energies)
print(facets_fractions)

#### Launch OptaDOS

#### Get QE from OptaDOS output

#### Calculate the weighted average QE

#### Calculate and display plenty more properties (DOS, bands, ...)